### Required libraries :

In [ ]:
import pandas as pd  # dataframe structure
import numpy as np  # array structure
import seaborn as sns # visualization
import matplotlib.pyplot as plt # plot
import glob, os  # dealing with OS and Files, reading files etc 

### Loading data :

<mark>A noter j'ai supprimé les données concernant 2009 car n'a pas la même forme que les autres ( juste pour lire tout en même temps)</mark>

Dans un premier on va lire tous les fichiers csv qu'on a téléchargé à partir du site datagov :

####  Data set about caracteristiques :

In [ ]:
path =r'dataset/caracteristiques' # path containing all csv files about caracteristiques (  use your own path)
allFiles = glob.glob(path + "/*.csv")  # a list containing files names (all csv files)
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,encoding='latin-1') # read csv file (file_)
    list_.append(df)  # append the the dataframe df to the lust list_
caracteristiques = pd.concat(list_)  # concat all dataframes existing in list_

#####  Data set about usagers :

In [ ]:
path =r'dataset/usagers' 
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,encoding='latin-1')
    list_.append(df)
usagers = pd.concat(list_)

####  Data set about lieux :

In [ ]:
path =r'dataset/lieux' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,encoding='latin-1')
    list_.append(df)
lieux = pd.concat(list_)

#### Data set about vehicules :

In [ ]:
path =r'dataset/vehicules' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,encoding='latin-1')
    list_.append(df)
vehicules = pd.concat(list_)

##### Short description about our dataframes :

In [ ]:
print("==== Shape of our dataframes ====")
print("The shape of caracteristiques is :", caracteristiques.shape)
print("The shape of lieux is :", lieux.shape)
print("The shape of vehicules is :", vehicules.shape)
print("The shape of usagers is :", usagers.shape)
print("==== Count Missing Data (note that 0 means also a nan value but should be converted to nan later during cleaning stage)  ====")
print("Total missing data in  caracteristiques is :", caracteristiques.isnull().sum().sum())
print("Total missing data in  lieux is :", lieux.isnull().sum().sum())
print("Total missing data in  vehicules is :", vehicules.isnull().sum().sum())
print("Total missing data in  usagers is :", usagers.isnull().sum().sum())

### Tâche 1 : Prédiction de la gravité des accidents

Dans cette partie nous allons essayer d'effectuer un apprentissage supervisé sur la variable : gravité de l'accident

Nous ne prenons pas en compte les variables spatiales. Donc nous ne faisons pas de nettoyage pour ces variables pour cette première tâche, mais nous allons nettoyer les autres variables ( imputation des valeurs manquantes, les points abberrants)

On va nettoyer les quatre tables qu'on vient de récupérer ci-dessus. Chaque table contient l'historique de 2005 jusqu'à 2016 ( sauf 2009).
Une fois les tables sont bien nettoyées, on va effectuer une jointure afin d'avoir toutes les variables nécessaires pour effectuer notre classification. La table résultante sera à la fois notre jeu de données sur toute la période(2005-2016) et  contenant les sous jeu de données correspondants à chaque année.
Avant de commencer la modélisation, nous allons effectuer des explorations ainsi que des analyses statistiques visant d'une part à visualiser les données et d'autre part à comprendre le jeu de donnée. Cette étape est primordiale car il constitue une étape de la méthode CRISP -largement utilisée dans le monde industrielle- utilisée en data mining.

### Data cleaning  :

---
####  La table caracteristiques 
---

In [ ]:
import missingno as msno
# ici on prend en compte que les nan
msno.matrix(caracteristiques)

###### gps :

In [ ]:

print(len(caracteristiques.gps.unique()))
caracteristiques.gps.unique()
#On voit que gps a 8 valeurs uniques, alors que dans le documents pdf il n'y a que 5 ( M,A,G,R et Y)

nan, 0 et S se sont considérées comme des valeurs manquantes

In [ ]:
caracteristiques[caracteristiques.gps=='S'].shape

In [ ]:
caracteristiques[caracteristiques.gps=='0'].shape

In [ ]:
caracteristiques.gps.isnull().sum()

In [ ]:
# puisque cette variable est utilisée pour localiser l'accident, on va remplacer '0' et'S' par nan.
caracteristiques.loc[:,['gps']]=caracteristiques.loc[:,['gps']].replace('0',np.nan)
caracteristiques.loc[:,['gps']]=caracteristiques.loc[:,['gps']].replace('S',np.nan)

In [ ]:
#verification :
print(len(caracteristiques.gps.unique()))
caracteristiques.gps.unique()

In [ ]:
caracteristiques.gps=caracteristiques.gps.astype(str) # on change ty type car nan est float alors que la variable est un object

In [ ]:
caracteristiques[caracteristiques.gps=='nan'].head() # pour voir le lien entre gps et dep

In [ ]:
caracteristiques[caracteristiques.gps=='nan'].tail() # pour voir le lien ente gps et dep

On peut remarquer que il y a des nan dans la france metropole et la france d'outre mer.

-Donc pour France Metropole on remplace gps par M ( il suffit de voir dep<950)

-Donc pour la france d'outre mer il suffit de voir aussi dep.

In [ ]:
# France M :
caracteristiques.ix[caracteristiques.dep<=950,'gps']=caracteristiques.ix[caracteristiques.dep<=950,'gps'].replace('nan','M')

In [ ]:
caracteristiques.ix[caracteristiques.dep==971,'gps']=caracteristiques.ix[caracteristiques.dep==971,'gps'].replace('nan','A')
caracteristiques.ix[caracteristiques.dep==972,'gps']=caracteristiques.ix[caracteristiques.dep==972,'gps'].replace('nan','A')
caracteristiques.ix[caracteristiques.dep==973,'gps']=caracteristiques.ix[caracteristiques.dep==973,'gps'].replace('nan','G')
caracteristiques.ix[caracteristiques.dep==974,'gps']=caracteristiques.ix[caracteristiques.dep==974,'gps'].replace('nan','R')
caracteristiques.ix[caracteristiques.dep==976,'gps']=caracteristiques.ix[caracteristiques.dep==976,'gps'].replace('nan','Y')

Donc cette variable est bien imputée.
Il reste juste le fait de fixer le type de cette variable. Elle est déjà 'object'.

###### jour :

In [ ]:
print(len(caracteristiques.jour.unique()))
caracteristiques.jour.unique()
# pas de soucis pour cette variable

###### mois :

In [ ]:
print(len(caracteristiques.mois.unique()))
caracteristiques.mois.unique()
# pas de soucis pour cette variable


##### an :

In [ ]:
print(len(caracteristiques.an.unique()))
caracteristiques.an.unique()
#pareil

###### hrmn: 

In [ ]:
caracteristiques.hrmn.min(), caracteristiques.hrmn.max()
# donc ici il s'agit de l'heure et les minutes
# exemple : 2359 ==> 23:59
# on cherche le len des éléments 
mes_len=[len(ele) for ele in set(list(caracteristiques.hrmn.astype(str).values))]
[ele for ele in set(mes_len)]
# Il y a donc différents len ( de 1 ==>4) 
# donc if faut du traitement
# si len == 4 ==> ajouter : entre les deux chiffres
# si len == 3 ==> ajouter 0 en avant puis : entre les deux chiffres
# si len == 2 ==> ajouter deux 00 vers la fin puis : 
# si len == 1 ==> ajouter 0 avant et 00 à la fin puis : 

In [ ]:
# change type to str for easy handling :
caracteristiques.hrmn=caracteristiques.hrmn.astype(str)

In [ ]:
# len ==4 : split with len=2 then add :
caracteristiques.ix[caracteristiques.hrmn.str.len()==4,'hrmn']= caracteristiques.ix[caracteristiques.hrmn.str.len()==4,'hrmn'].str.extract('(.{2,2})' * 2).apply(lambda x: ':'.join(x), axis=1)

In [ ]:
# len == 3 :  add 0 to into the start then split then add :
caracteristiques.ix[caracteristiques.hrmn.str.len()==3,'hrmn']=('0'+caracteristiques.ix[caracteristiques.hrmn.str.len()==3,'hrmn']).str.extract('(.{2,2})' * 2).apply(lambda x: ':'.join(x), axis=1)

In [ ]:
#len =2 : add 00 to the end then split then add : 
caracteristiques.ix[caracteristiques.hrmn.str.len()==2,'hrmn']=(caracteristiques.ix[caracteristiques.hrmn.str.len()==2,'hrmn'] +'00').str.extract('(.{2,2})' * 2).apply(lambda x: ':'.join(x), axis=1)

In [ ]:
# len == 1 ==> ajouter 0 avant et 00 à la fin puis : 
caracteristiques.ix[caracteristiques.hrmn.str.len()==1,'hrmn']=('0'+caracteristiques.ix[caracteristiques.hrmn.str.len()==1,'hrmn'] +'00').str.extract('(.{2,2})' * 2).apply(lambda x: ':'.join(x), axis=1)

In [ ]:
# verification :
mes_len_apres=[len(ele) for ele in set(list(caracteristiques.hrmn.astype(str).values))]
[ele for ele in set(mes_len_apres)]
# c'est bon ( len==5 car on a ajouté les :)

###### lum :

In [ ]:
print(len(caracteristiques.lum.unique()))
caracteristiques.lum.unique()
# bon

In [ ]:
# change type to object
caracteristiques.lum=caracteristiques.lum.astype(str)

###### agg


In [ ]:
print(len(caracteristiques['agg'].unique()))
caracteristiques['agg'].unique()
#bon

In [ ]:
caracteristiques.loc[:,['agg']]=caracteristiques.loc[:,['agg']].astype(str)

###### intersection :

In [ ]:
print(len(caracteristiques.int.unique()))
caracteristiques.int.unique()

In [ ]:
# 0 est un intrus 
caracteristiques[caracteristiques.int==0].shape
# il y 106 valeurs de 0
# on va remplacer cette valeur par la valeur la plus courante 

In [ ]:
caracteristiques.int.astype(str).describe()
# le plus courant c'est 1 avec une fréquence de 547440
# c'est normale car la majorité des accidents n'ont pas lieu sur les intersections ( vitesse très faible générelement)

In [ ]:
# remplacer 0 par 1: 
caracteristiques.loc[:,['int']]=caracteristiques.loc[:,['int']].replace(0,1)

In [ ]:
#verifications :
print(len(caracteristiques.int.unique()))
caracteristiques.int.unique()

In [ ]:
caracteristiques.int=caracteristiques.int.astype(str)

###### atm :

In [ ]:
print(len(caracteristiques.atm.unique()))
caracteristiques.atm.unique()
# il y des nan values, de même on va les remplacer par la valeur la plus courante

In [ ]:
# taille des nan values dans atm :
print('taille :', caracteristiques.atm.isnull().sum())
# valeur la plus courante dans atm :
caracteristiques.atm.astype(str).describe()
# 1 est la valeur la plus courante ( normale)

In [ ]:
#remplacer nan par 1 :
caracteristiques.loc[:,['atm']]=caracteristiques.loc[:,['atm']].replace(np.nan,1)

In [ ]:
#verification :
print(len(caracteristiques.atm.unique()))
caracteristiques.atm.unique()

In [ ]:
caracteristiques.atm=caracteristiques.atm.astype(str)

###### collision :

In [ ]:
print(len(caracteristiques.col.unique()))
caracteristiques.col.unique()
# nan values are in 

In [ ]:
# taille des nan values dans col :
print('taille :', caracteristiques.col.isnull().sum())
# valeur la plus courante dans col :
caracteristiques.col.astype(str).describe()
# 6 est la valeur la plus courante ( autre collision)

In [ ]:
#remplacer nan par 6 :
caracteristiques.loc[:,['col']]=caracteristiques.loc[:,['col']].replace(np.nan,6)

In [ ]:
#verification :
print(len(caracteristiques.col.unique()))
caracteristiques.col.unique()

In [ ]:
caracteristiques.col=caracteristiques.col.astype(str)

###### Adresse postale :

In [ ]:
print(len(caracteristiques.adr.unique()))
caracteristiques.adr.unique()
# des adresse, donc pas de problémes ( sauf erreur de saisie. on va lire adresse par adresse bien sûr pour s'assurer )

##### Variables numériques :

In [ ]:
# valeur nan par variable
caracteristiques.isnull().sum()

###### lon/lat

 lat et lon sont des variables numériques. Ils présentent plus de 50% de valeurs manquantes. On les garde comme ça pour l'instant.

###### com :

Chaque dépertement contient des communes. Dans notre base les déparetements sont chiffrés entre 10 et 976. 

RQ :
    - on peut aggréger nos données par département et étudier chaque département tout seul
    - On peut encore ajouter de la granulité et aggréger par commune.


In [ ]:
#nan ==> le mode
caracteristiques.com.astype(str).describe()

In [ ]:
caracteristiques.loc[:,['com']]=caracteristiques.loc[:,['com']].replace(np.nan,55)

In [ ]:
# change type to object :
caracteristiques.com=caracteristiques.com.astype(str)

###### dep

In [ ]:
#pas de valeur manquante pour cette variables
print("il y a ",len(caracteristiques.dep.unique()),"départements ")
print(sorted(list(caracteristiques.dep.unique())))


In [ ]:
dep_values=list(caracteristiques.dep.astype(str).values)

In [ ]:
set([dep for dep in dep_values if len(dep)==2])
# on a bien toutes les valeurs correspondantes au dépertement de numéro entre 1 et 9.

In [ ]:
print(set([dep for dep in dep_values if len(dep)==3 and dep[-1]!='0' and dep!='202' and dep!='201'])) 

In [ ]:
# 201 et 202 remplacent 2A et 2B ( voir lien www.francegene.com/rech-fr/dep-fr.php) : donc pour ces deux valeurs pas de soucis
# mais les autres ils ne se terminent pas par 0 ?
#  normalement il y a 95 departement  d'après le lien ci-dessous, donc 950 doit être la plus grande valeur.
# A voir : soit se sont des nouveau dépertement ? sinon on va les remplacer par la plus courante valeurs.
# en fait les autres numéros sont celles de la france d'outre mer. plus précisement :
# 971 : num départemental de Guadeloupe
# 972 : Maritinique
# 973 : La Guyane 
# 974 : La Réunion
# 976 : Mayotte
# Donc c'est à nous de choisir : est ce que juste la france europééne ou aussi la france d'outre mer.

In [ ]:
#change type to str
caracteristiques.dep=caracteristiques.dep.astype(str)

In [ ]:
#save table to csv file :
caracteristiques.to_csv('caracteristiques_2005-2016.csv',index=False,)

---
#### la table lieux : 
---

Commençons par un graphiques montrant la distribution des valeurs manquantes dans cette table.

In [ ]:
#!pip install missingno # if you do not have this package try installing it using the given command

In [ ]:
import missingno as msno
msno.matrix(lieux)

Comme vous voyez il y a des variables qui ont beaucoup de valeurs manquantes( ceci sans compter les valeurs '0'), surtout : v1,v2, pr et pr1.

Nous allons maintenant générer un heatmap de corrélation qui sert à montrer une distribution conditionnelle des valeurs manquantes entre les différentes variables :

In [ ]:
msno.heatmap(lieux)

Une carte de chaleur de corrélation simple est montrée ci-dessus. Cette carte décrit le degré de relation de nullité entre les différentes variables. La plage de valeurs de cette corrélation de nullité va de -1 à 1 (-1 ≤ R ≤ 1). Les entités sans valeur manquante sont exclues du heatmap (par exemple: Num_Acc). Si la corrélation de nullité est très proche de zéro (-0,05 <R <0,05) (voie et catr), aucune valeur ne sera affichée. En outre, une corrélation de nullité positive parfaite (R = 1) indique que la première variable et la deuxième variable ont toutes deux des valeurs manquantes correspondantes (MAR), alors qu'une corrélation de nullité négative parfaite (R = -1) signifie que l'une des variables est manquante et la seconde n'est pas manquante (MNAR).

Par conséquent, vu que la majorité des valeurs de R sont importantes, on va considérer que les valeurs manquantes sont des MAR ( Missing At Random). Ceci a pour objectif de trouver la bonne méthode d'imputation des valeurs manquantes. Certe ici, la bonne méthode sera par exemple <b>MICE</b> (<b> Multiple Imputation by Chained Equations</b>)

Mais cette méthode s'applique dans le cas où les variables sont numériques. Pour cela on se contente d'utiliser une méthode stationnaire d'imputation: imputation par le plus fréquent.

Avant d'appliquer cette méthode, il faut nettoyer les cases avec les valeurs '0'. Ceci est important car ce n'est pas évident de remplacer les 0 par des nan simplement, puisque parfois '0' est une vraie valeurs à garder ( on verra des exemples de variables où ceci est vrai).

###### catégorie de route :

In [ ]:
print("Les valeurs uniques existantes dans la variable catr sont : \n",lieux.catr.unique())
# il y a nan dans cette série, donc nous devons les imputer

In [ ]:
# 9 spécifie "autre"  type de route
print("Il y a {} valeurs de 9 dans la variable catr".format(lieux[lieux.catr==9].shape))  
#taille valeur nan
print("Il y a {} valeurs nan dans la variable catr ".format(lieux.catr.isnull().sum()))

In [ ]:
# remplacer nan par 9.( 9 toujours n'est pas connue donc implicitement c'est nan)
lieux.loc[:,['catr']]=lieux.loc[:,['catr']].replace(np.nan,9)

###### régime circulation :

In [ ]:
print("Les valeurs uniques existantes dans la variable circ sont : ",lieux.circ.unique())
# il y a 0 et nan qui ne sont pas des vraies valeurs pour cette variable

In [ ]:
print("taille 0 : ",lieux[lieux.circ==0].shape[0])
print("taille nan : ",lieux.circ.isnull().sum())
# on remplace les 0 par nan

In [ ]:
# on cherche le mode puis on impute nan et 0 par ce mode
#print(float(lieux.circ.astype(str).describe().top))  # donne le mode de la série
# top est 2( les accidents ont lieu beaucoup sur les  routes nationals)

In [ ]:
lieux.loc[:,['circ']]=lieux.loc[:,['circ']].replace(np.nan,float(lieux.circ.astype(str).describe().top))
lieux.loc[:,['circ']]=lieux.loc[:,['circ']].replace(0,float(lieux.circ.astype(str).describe().top))

###### voie réservée :

In [ ]:
print("Il y a {} comme valeurs uniques dans  la variable vosp".format(lieux.vosp.unique()))
# 0 et nan sont à corriger

In [ ]:
print("taille 0 :",lieux[lieux.vosp==0].shape)
print("taille nan :",lieux.vosp.isnull().sum())
# on remarque que quasiment toute la colonne est en '0'.
# on garde 0 ( car la variable 0 ici a pour signification : pas de voie réservèe)
 

In [ ]:
lieux.loc[:,['vosp']]=lieux.loc[:,['vosp']].replace(np.nan,0)


###### profil de la route : 

In [ ]:
print("Les valeurs uniques pour la varible prof sont : ",lieux.prof.unique())

In [ ]:
print("taille 0 :",lieux[lieux.prof==0].shape)
print("taille nan :",lieux.prof.isnull().sum())

In [ ]:
lieux.loc[:,['prof']]=lieux.loc[:,['prof']].replace(0,float(lieux.prof.astype(str).describe().top))
lieux.loc[:,['prof']]=lieux.loc[:,['prof']].replace(np.nan,float(lieux.prof.astype(str).describe().top))

###### Tracé en plan :
    
    

In [ ]:
print("les valeurs uniques de la variable plan sont :",lieux.plan.unique())

In [ ]:
print("taille 0 :",lieux[lieux.plan==0].shape)
print("taille nan :",lieux.plan.isnull().sum())

In [ ]:
# on remplace 0 par nan :
lieux.plan.astype(str).describe()
# top est 1 ( Partie rectiligne)

In [ ]:
lieux.loc[:,['plan']]=lieux.loc[:,['plan']].replace(0,lieux.plan.astype(str).describe().top)
lieux.loc[:,['plan']]=lieux.loc[:,['plan']].replace(np.nan,lieux.plan.astype(str).describe().top)

###### Etat de la surface 

In [ ]:
print("Les valeurs uniques pour la variable surf sont : ",lieux.surf.unique())


In [ ]:
print("taille 0 :",lieux[lieux.surf==0].shape)
print("taille nan :",lieux.surf.isnull().sum())

In [ ]:
# on remplace par 9 ( car 9 veut dire autre autre)
lieux.loc[:,['surf']]=lieux.loc[:,['surf']].replace(0,9)
lieux.loc[:,['surf']]=lieux.loc[:,['surf']].replace(np.nan,9)

###### situation de l'acccident

In [ ]:
print("les valeurs uniques pour la variable situ sont :", lieux.situ.unique())

In [ ]:
print("taille 0 :",lieux[lieux.situ==0].shape)
print("taille nan :",lieux.situ.isnull().sum())

In [ ]:
# on remplace les 0 par nan :
lieux.situ.astype(str).describe()
#top est 1 ( sur chaussé)

In [ ]:
lieux.loc[:,['situ']]=lieux.loc[:,['situ']].replace(0,float(lieux.situ.astype(str).describe().top))
lieux.loc[:,['situ']]=lieux.loc[:,['situ']].replace(np.nan,float(lieux.situ.astype(str).describe().top))

###### point école :

In [ ]:
# on sait pas pour cette variables quelles sont les veleurs prises
print("les valeurs uniques pour la variable env1 sont :",lieux.env1.unique())
# on laisse 0 comme valeur ( à voir ..)

###### voie :

In [ ]:
print("les valeurs uniques pour la variable voie sont :", lieux.voie.unique())

In [ ]:
print("total des nan dans voie :",lieux.voie.isnull().sum())

In [ ]:
lieux.voie.describe()
# contient beaucoup de 0 alors que 0 n'est pas un numéro de route 
# on transforme 0 à nan 

In [ ]:
lieux.loc[:,['voie']]=lieux.loc[:,['voie']].replace(0,np.nan)


In [ ]:
lieux.loc[:,['voie']]=lieux.loc[:,['voie']].replace(np.nan,int(lieux.voie.describe().top))

###### Indice numériques du numéro de la route : 

In [ ]:
print("Les valeurs uniques pour la variable v1 sont :",lieux.v1.unique())

In [ ]:
# on remplace 0 par nan :
lieux.loc[:,['v1']]=lieux.loc[:,['v1']].replace(0,int(lieux.v1.mean()))
lieux.loc[:,['v1']]=lieux.loc[:,['v1']].replace(np.nan,int(lieux.v1.mean()))

###### Lettre V2

In [ ]:
print("Les valeurs uniques pour la variable v2 sont :", lieux.v2.unique())
# il y a un probléme de encoding : /x04
# il y des nan 
# il y des 0  
# il faut comprendre la variable pour l'imputer.
# C'est juste un indice, on peut éliminer cette variable et garder que voie et v1

In [ ]:
lieux=lieux.drop('v2',axis=1) # delete v2 from dataframe

###### nombre total de voie de circulation  :

In [ ]:
print(sorted(set(list(lieux.nbv.unique()))))
# il y a des nan 

In [ ]:
# variable numérique==> on remplace par la moyenne
lieux.loc[:,['nbv']]=lieux.loc[:,['nbv']].replace(np.nan,int(lieux.nbv.mean())) #  on prends la partie entiére

###### Numéro PR :
    

In [ ]:
#de même variable numérique
lieux.loc[:,['pr']]=lieux.loc[:,['pr']].replace(np.nan,(lieux.pr.mean())) 

###### Distance en mètres au PR :

In [ ]:
# variable numérique

lieux.loc[:,['pr1']]=lieux.loc[:,['pr1']].replace(np.nan,lieux.pr1.mean()) 

###### Largeur du terre plein central TPC ( s il exist )

In [ ]:
# variable  numérique

lieux.loc[:,['lartpc']]=lieux.loc[:,['lartpc']].replace(np.nan,lieux.lartpc.mean()) 

###### larrout

In [ ]:
# variable numérique

lieux.loc[:,['larrout']]=lieux.loc[:,['larrout']].replace(np.nan,lieux.larrout.mean())

###### infra

In [ ]:
print("les valeurs uniques dans la variable infra sont :",lieux.infra.unique())

In [ ]:
lieux.infra.astype(str).describe()
# on garde 0 comme étant une autre valeur.
# on remplace nan par 0
lieux.loc[:,['infra']]=lieux.loc[:,['infra']].replace(np.nan,float(lieux.infra.astype(str).describe().top))

On vient de finir le nettoyage de cette table. Il reste à donner le bon type pour chaque variable.

In [ ]:
# save table to csv file
lieux.to_csv('lieux_2005-2016.csv',index=False)

---
#### la table usagers 
---

In [ ]:
import missingno as msno
msno.matrix(usagers)

###### place : 

In [ ]:
usagers.place.unique()

In [ ]:
print("taille 0 :" , usagers[usagers.place==0].shape)
print("taille nan :" , usagers.place.isnull().sum())


In [ ]:
#on les remplace par ? 
# ici les valeurs manquantes vont dependre de la nature du vehicule 
# donc pour chaque type de vehicule on va remplacer les valeurs manquantes par le mode.
# donc on verra cette variable apès avoir fait la jointure entre les tables 
#sinon on peut simplifier et remplacer par le mode car dans tous les cas le mode sera la place 1 du conducteur
usagers.place.astype(str).describe()

In [ ]:
usagers.loc[:,['place']]=usagers.loc[:,['place']].replace(0,1)
usagers.loc[:,['place']]=usagers.loc[:,['place']].replace(np.nan,1)

###### Catégorie d'usager :

In [ ]:
usagers.catu.unique()
#c'est bon

###### Gravité de l'accident​ :

In [ ]:
usagers.grav.unique()
#c'est bon

###### Sexe de l'usager :


In [ ]:
usagers.sexe.unique()
#c'est bon

###### Annee de naissance de l'usager'

In [ ]:
usagers.an_nais.dtype
# pas de soucis ici sauf si il y une erreur dans la saisie  d'une data :
print('min des dates :',usagers.an_nais.min())
print('max des dates :',usagers.an_nais.max())

In [ ]:
# nan values :
usagers.an_nais.isnull().sum()

In [ ]:
usagers.loc[:,['an_nais']]=usagers.loc[:,['an_nais']].replace(np.nan,int(usagers.an_nais.mean()))

In [ ]:
#verification :
print('min des dates :',usagers.an_nais.min())
print('max des dates :',usagers.an_nais.max())
# c'est bon la moyenne est comprise entre max et mean

###### Motif du déplacement au moment de l’accident :

In [ ]:
usagers.trajet.unique()

In [ ]:
print("taille 0 :" , usagers[usagers.trajet==0].shape)
print("taille nan :" , usagers.trajet.isnull().sum())

In [ ]:
# on remplace par 9 :
usagers.loc[:,['trajet']]=usagers.loc[:,['trajet']].replace(0,9)
usagers.loc[:,['trajet']]=usagers.loc[:,['trajet']].replace(np.nan,9)

###### secu :

In [ ]:
usagers.secu.unique()

In [ ]:
secu_values=list(usagers.secu.astype(str).values)


In [ ]:
# on traite d'abord le len puis les nan :
secu_values_set=[secu[:-2] for secu in secu_values if secu!='nan']
# d'après le document chaque valeur est censée d'avoir deux chiffres:

In [ ]:
print(set(secu_values_set))
# il y le 0 et parfois on a qu'un seul chiffre ( ex 2, 1, 3)

In [ ]:
usagers.secu.astype(str).describe()
#top 11 : ceinture exist et elle est utilisée

In [ ]:
# remplacer nan et 0 par 11 :
usagers.loc[:,['secu']]=usagers.loc[:,['secu']].replace(0,11)
usagers.loc[:,['secu']]=usagers.loc[:,['secu']].replace(np.nan,11)

In [ ]:
# pour le reste : soit on simplifie et on remplace par 11
# soit on compléte le chiffre par un deuxième qui soit le mode approprié au chiffre existant :
# deuxième méthode est plus logique :
from statistics import mode
secu_1=[secu for secu in secu_values if '1' in secu]
mode(secu_1)
# donc on remplace par 11


In [ ]:
secu_2=[secu for secu in secu_values if '2' in secu]
mode(secu_2)
# on remplace par '21'

In [ ]:
secu_3=[secu for secu in secu_values if '3' in secu]
mode(secu_3)
# on remplace par '13'

In [ ]:
usagers.loc[:,['secu']]=usagers.loc[:,['secu']].replace(1,11)
usagers.loc[:,['secu']]=usagers.loc[:,['secu']].replace(2,21)
usagers.loc[:,['secu']]=usagers.loc[:,['secu']].replace(3,13)


###### Localisation du piéton :

In [ ]:
usagers.locp.unique()

In [ ]:
print("taille 0 :" , usagers[usagers.locp==0].shape)
print("taille nan :" , usagers.locp.isnull().sum())
print(usagers.shape)

In [ ]:
# ici on va laisser 0 comme une autre valeur, il se peut qu'elle aie pour signification : 'il n'y a pas de piéton'
# on remplace nan par 0 également 
usagers.loc[:,['locp']]=usagers.loc[:,['locp']].replace(np.nan,0)

###### Action du piéton :

In [ ]:
usagers.actp.unique()

In [ ]:
# nan ==> 0
usagers.loc[:,['locp']]=usagers.loc[:,['locp']].replace(np.nan,0)

###### etatp

In [ ]:
usagers.etatp.unique()

In [ ]:
# on laisse 0 comme une autre valeur qui va indiquer qu'il n'y avait pas de piéton
# nan ==> 0
usagers.loc[:,['etatp']]=usagers.loc[:,['etatp']].replace(np.nan,0)

On vient de finir le nettoyage de cette table. Il reste à changer les types :

In [ ]:
usagers.to_csv('usager_2005-2016.csv', index=False)

---
la table des vehicules
---

In [ ]:
import missingno as msno
msno.matrix(vehicules)

###### Sens de circulation

In [ ]:
vehicules.senc.unique()

In [ ]:
print("taille 0 :" , vehicules[vehicules.senc==0].shape)
print("taille nan :" , vehicules.senc.isnull().sum())

In [ ]:
vehicules.senc.astype(str).describe()

In [ ]:
# on remplace par une valeur au choix :  sens croissant 
vehicules.loc[:,['senc']]=vehicules.loc[:,['senc']].replace(np.nan,1)
vehicules.loc[:,['senc']]=vehicules.loc[:,['senc']].replace(0,1)

###### Catégorie du véhicule :

In [ ]:
print(len(vehicules.catv.unique()))
vehicules.catv.unique()
#Ok

###### Obstacle mobile heurté :

In [ ]:
vehicules.obsm.unique()

In [ ]:
# nan,0===>9
vehicules.loc[:,['obsm']]=vehicules.loc[:,['obsm']].replace(np.nan,9)
vehicules.loc[:,['obsm']]=vehicules.loc[:,['obsm']].replace(0,9)

###### Point de choc initial :

In [ ]:
vehicules.choc.unique()

In [ ]:
# 0, nan ==> mode :
vehicules.choc.astype(str).describe()

In [ ]:
vehicules.loc[:,['choc']]=vehicules.loc[:,['choc']].replace(np.nan,1)
vehicules.loc[:,['choc']]=vehicules.loc[:,['choc']].replace(0,1)

###### Nombre d’occupants dans le transport en commun

In [ ]:
vehicules.occutc.unique()
# 0 pour dire que ce n'est pas transport commun.

###### manv

In [ ]:
# manv de type int
vehicules.manv.isnull().sum()

In [ ]:
vehicules.manv.astype(str).describe()

In [ ]:
vehicules.loc[:,['manv']]=vehicules.loc[:,['manv']].replace(np.nan,1)

In [ ]:
vehicules.dtypes

save to csv file 

In [ ]:
vehicules.to_csv('vehicules_2005-2016.csv', index=False)

---
Fin nettoyage 
---

Dans le but d'une bonne gestion de mémoire on va élimner toutes les variables, puis on récupére nos tables déjà nettoyées.

### La jointure des tables :

In [40]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


On commence par récupérer les données qu'on a déjà enregistré localement puis changer le type des variables:

In [41]:
import pandas as pd
#load caracteristiques and change types:
caracteristiques=pd.read_csv('caracteristiques_2005-2016.csv', encoding='latin-1')
caracteristiques.jour=caracteristiques.jour.astype(str)
caracteristiques.mois=caracteristiques.mois.astype(str)
caracteristiques.lum=caracteristiques.lum.astype(str)
caracteristiques.loc[:,['agg']]=caracteristiques.loc[:,['agg']].astype(str)
caracteristiques.int=caracteristiques.int.astype(str)
caracteristiques.atm=caracteristiques.atm.astype(str)
caracteristiques.col=caracteristiques.col.astype(str)


#load lieux and change types:
lieux=pd.read_csv('lieux_2005-2016.csv',encoding='latin-1')
lieux.catr=lieux.catr.astype(str)
lieux.circ=lieux.circ.astype(str)
lieux.vosp=lieux.vosp.astype(str)
lieux.prof=lieux.prof.astype(str)
lieux.surf=lieux.surf.astype(str)
lieux.infra=lieux.infra.astype(str)
lieux.situ=lieux.situ.astype(str)

# load vehicules and change types: 
vehicules=pd.read_csv('vehicules_2005-2016.csv',encoding='latin-1')
vehicules.senc=vehicules.senc.astype(str)
vehicules.catv=vehicules.catv.astype(str)
vehicules.obs=vehicules.obs.astype(str)
vehicules.obsm=vehicules.obsm.astype(str)
vehicules.choc=vehicules.choc.astype(str)
vehicules.manv=vehicules.manv.astype(str)

# load usagers and change types:
usagers=pd.read_csv('usager_2005-2016.csv',encoding='latin-1')
usagers.place=usagers.place.astype(str)
usagers.catu=usagers.catu.astype(str)
usagers.grav=usagers.grav.astype(str)
usagers.sexe=usagers.sexe.astype(str)
usagers.trajet=usagers.trajet.astype(str)
usagers.secu=usagers.secu.astype(str)
usagers.locp=usagers.locp.astype(str)
usagers.actp=usagers.actp.astype(str)
usagers.etatp=usagers.etatp.astype(str)

C:\Users\rlouriz\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On merge caracteristiques et lieux puis on merge avec vehicules : 

In [42]:
carac_lieux_veh_merged=pd.merge(pd.merge(caracteristiques,lieux,on='Num_Acc'), vehicules,on='Num_Acc' )

on merge finalement avec usagers :

In [43]:
data_merged=pd.merge(carac_lieux_veh_merged,usagers,on=['Num_Acc','num_veh'])

In [44]:
#save data_merged into a csv file : 
data_merged.to_csv('data_merged.csv',index=False)

---
# Model de prédiction de la gravité des accidents avec toutes les données (2005-2016) :
---

Dans cette partie, nous allons effectuer un modèle de classification afin de prédire la gravité d'un accident en utilisant tout l'historique dont on dispose : 

In [45]:
# let's first create an attribute age :
data_merged.loc[:,['an']]=data_merged.loc[:,['an']]+2000
data_merged['age']=pd.Series(data_merged.an.values - data_merged.an_nais.values)
data_merged['age']=data_merged['age'].astype(int)

In [66]:
data_merged.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,age
0,200500000001,2005,1,12,19:00,3,2,1,1.0,3.0,...,1,4,1,1.0,11.0,0.0,0.0,0.0,1976.0,29.0
1,200500000001,2005,1,12,19:00,3,2,1,1.0,3.0,...,1,3,2,3.0,11.0,0.0,0.0,0.0,1968.0,37.0
2,200500000001,2005,1,12,19:00,3,2,1,1.0,3.0,...,2,1,1,9.0,11.0,0.0,0.0,0.0,1964.0,41.0
3,200500000001,2005,1,12,19:00,3,2,1,1.0,3.0,...,2,1,1,9.0,31.0,0.0,0.0,0.0,2004.0,1.0
4,200500000001,2005,1,12,19:00,3,2,1,1.0,3.0,...,2,1,1,9.0,11.0,0.0,0.0,0.0,1998.0,7.0


In [114]:
print(data_merged.columns)
# columns used for classification : we detele all spatial features and also temporalle one 
col_used=[ 'lum', 'agg', 'int', 'atm','col',  'catr',
       'circ', 'vosp', 'prof',
        'surf', 'infra', 'situ', 'senc', 'catv','obsm', 'choc', 'manv', 'place', 'catu', 'grav',
        'sexe', 'trajet', 'secu', 'locp', 'actp', 'etatp']

Index(['Num_Acc', 'an', 'mois', 'jour', 'hrmn', 'lum', 'agg', 'int', 'atm',
       'col', 'com', 'adr', 'gps', 'lat', 'long', 'dep', 'catr', 'voie', 'v1',
       'circ', 'nbv', 'pr', 'pr1', 'vosp', 'prof', 'plan', 'lartpc', 'larrout',
       'surf', 'infra', 'situ', 'env1', 'senc', 'catv', 'occutc', 'obs',
       'obsm', 'choc', 'manv', 'num_veh', 'place', 'catu', 'grav', 'sexe',
       'trajet', 'secu', 'locp', 'actp', 'etatp', 'an_nais', 'age'],
      dtype='object')


if you have a windows machine and want to install xgboost, just follow instructiosn in this link :
https://medium.com/@rakshithvasudev/how-i-installed-xgboost-after-a-lot-of-hassels-on-my-windows-machine-c53e972e801e

In [115]:
#import xgboost and evaluations libraries
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# data for training  and testing :
data=data_merged.loc[:,col_used]

In [116]:
X,y=data.loc[:,data.columns!='grav'], data.loc[:,'grav']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

In [ ]:

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrixxgboost_model=XGBClassifier()
#fit model on training set
xgboost_model.fit(X_train,y_train)
#predict on test set :
y_pred=xgboost_model.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
print("Confsuion matrix : \n ",cm)
print('Classification report : \n ',classification_report(y_test,y_pred))
y_test.value_counts()



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train1, X_test1,Y_train1,Y_test1 = train_test_split(X, Y, test_size=0.33, random_state=99)
#Without weather
svc = SVC()
svc.fit(X_train1, Y_train1)
Y_pred = svc.predict(X_test1)
acc_svc1 = round(svc.score(X_test1, Y_test1) * 100, 2)
acc_svc1

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train1, Y_train1)
Y_pred = knn.predict(X_test1)
acc_knn1 = round(knn.score(X_test1, Y_test1) * 100, 2)
acc_knn1


# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train1, Y_train1)
Y_pred = logreg.predict(X_test1)
acc_log1 = round(logreg.score(X_train1, Y_train1) * 100, 2)
acc_log1


# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train1, Y_train1)
Y_pred = gaussian.predict(X_test1)
acc_gaussian1 = round(gaussian.score(X_test1, Y_test1) * 100, 2)
acc_gaussian1

# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train1, Y_train1)
Y_pred = perceptron.predict(X_test1)
acc_perceptron1 = round(perceptron.score(X_test1, Y_test1) * 100, 2)
acc_perceptron1

# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train1, Y_train1)
Y_pred = linear_svc.predict(X_test1)
acc_linear_svc1 = round(linear_svc.score(X_test1, Y_test1) * 100, 2)
acc_linear_svc1

# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train1, Y_train1)
Y_pred = sgd.predict(X_test1)
acc_sgd1 = round(sgd.score(X_test1, Y_test1) * 100, 2)
acc_sgd1

# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train1, Y_train1)
Y_pred = decision_tree.predict(X_test1)
acc_decision_tree1 = round(decision_tree.score(X_test1, Y_test1) * 100, 2)
acc_decision_tree1

# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train1, Y_train1)
Y_pred = random_forest.predict(X_test1)
random_forest.score(X_train1, Y_train1)
acc_random_forest1 = round(random_forest.score(X_test1, Y_test1) * 100, 2)
acc_random_forest1

In [ ]:
print("Machine Learning algorithm scores without weather related conditions")
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc1, acc_knn1, acc_log1, 
              acc_random_forest1, acc_gaussian1, acc_perceptron1, 
              acc_sgd1, acc_linear_svc1, acc_decision_tree1]})
models.sort_values(by='Score', ascending=False)

In [117]:
# Confusion matrix with random forest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 1)
rf = RandomForestClassifier(random_state = 4)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)
print('Classification report: \n',classification_report(y_test,y_pred))
y_test.value_counts()

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

# Jointure année par année

In [118]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x000002583E24FEF0>>
Traceback (most recent call last):
  File "C:\Users\rlouriz\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [119]:
import pandas as pd
caracteristiques=pd.read_csv('caracteristiques_2005-2016.csv',encoding='latin-1')
vehicules=pd.read_csv('vehicules_2005-2016.csv',encoding='latin-1')
lieux=pd.read_csv('lieux_2005-2016.csv',encoding='latin-1')
usagers=pd.read_csv('usager_2005-2016.csv',encoding='latin-1')

C:\Users\rlouriz\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [120]:
dfs=[] # a list that will contain data for each year
col=[2005,2006,2007,2008,2010,2011,2012,2013,2014,2015,2016]
for year in col :
    c=caracteristiques.loc[caracteristiques['Num_Acc']//100000000==year] # c is a dataframe for year='year'
    v=vehicules.loc[vehicules['Num_Acc']//100000000==year]
    u=usagers.loc[usagers['Num_Acc']//100000000==year]
    l=lieux.loc[lieux['Num_Acc']//100000000==year]
    merge1=pd.merge(c,l,on='Num_Acc')
    merge2=pd.merge(merge1,v,on='Num_Acc')
    data=pd.merge(merge2,u,on=['Num_Acc','num_veh'])
    dfs.append(data)
    

    
    
    
    

In [121]:
# save all dataframes into a csvs files :
for index, item in enumerate(dfs, start = 1):
    item.to_csv('data'+str(index)+'.csv')

# Modèle de prédiction de la gravité des accidents par année :

##### Load data : 

In [ ]:
final_data=pd.DataFrame()
for i in range(1,11):
    df=pd.read_csv('data'+str(i)+'.csv',encoding='latin-1')
    df= df.loc[(np.isfinite(df.long) & np.isfinite(df.lat)),:]
    final_data=pd.concat([final_data,df])


    

##### Describe data :

In [ ]:
print("Shape of our data is :",final_data.shape)
print("We have {} columns in our data".format(len(final_data.columns)))

##### Save data as csv file:

In [ ]:
final_data.to_csv('data_cleaned.csv')

In [ ]:
%reset  # clean memory

##### Load csv file : datacleaned

In [ ]:
import pandas as pd
df=pd.read_csv('data_cleaned.csv',encoding='latin-1')


In [ ]:
df.head()

In [ ]:
train=df.loc[:,col_used]

In [ ]:
# write to csv file
train.to_csv("training.csv")

In [233]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [288]:
import pandas as pd
train=pd.read_csv('data1.csv',encoding='latin-1')

In [289]:
# let's first create an attribute age :
train.loc[:,['an']]=train.loc[:,['an']]+2000
train['age']=pd.Series(train.an.values - train.an_nais.values)

In [290]:
train.jour=train.jour.astype(str)
train.lum=train.lum.astype(str)
train.loc[:,['agg']]=train.loc[:,['agg']].astype(str)
train.int=train.int.astype(str)
train.atm=train.atm.astype(str)
train.col=train.col.astype(str)

train.catr=train.catr.astype(str)
train.circ=train.circ.astype(str)
train.prof=train.prof.astype(str)
train.surf=train.surf.astype(str)
train.infra=train.infra.astype(str)
train.situ=train.situ.astype(str)

train.senc=train.senc.astype(str)
train.catv=train.catv.astype(str)
train.obs=train.obs.astype(str)
train.obsm=train.obsm.astype(str)
train.choc=train.choc.astype(str)
train.manv=train.manv.astype(str)


train.place=train.place.astype(str)
train.catu=train.catu.astype(str)
train.grav=train.grav.astype(str)
train.sexe=train.sexe.astype(str)
train.trajet=train.trajet.astype(str)
train.secu=train.secu.astype(str)
train.locp=train.locp.astype(str)
train.actp=train.actp.astype(str)
train.etatp=train.etatp.astype(str)

In [291]:
col_used=[  'jour', 'lum', 'agg', 'int', 'atm', 'col',
           'catr', 'voie', 'circ', 'nbv', 'pr', 'pr1', 'prof', 'plan', 'lartpc', 'larrout', 'surf', 'infra','obs',
           'situ', 'senc', 'catv', 'occutc', 'obsm', 'choc', 'manv', 'place', 'catu', 'grav', 'sexe',
           'trajet','secu', 'locp', 'etatp','age']
col_used2=[   'lum',  'int', 'col',
             'circ', 'nbv', 'surf', 'infra',
           'situ', 'senc', 'catv', 'occutc',  'place', 'catu', 'grav', 'sexe',
           'trajet','secu', 'locp','age']
train=train.loc[:,col_used2]

In [292]:
Y=train.grav.values
X=train.loc[:,train.columns!='grav']


In [293]:
#get dummies :
X=pd.get_dummies(X,prefix_sep='_')

In [298]:
len(list(X.columns))

122

In [299]:

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
xgboost_model=XGBClassifier()
#fit model on training set
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=1)
xgboost_model.fit(X_train,y_train)
#predict on test set :
y_pred=xgboost_model.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
print("Confsuion matrix : \n ",cm)
print('Classification report : \n ',classification_report(y_test,y_pred))

Confsuion matrix : 
  [[20866     1   435  2902]
 [  549    53   527   502]
 [ 3764    45  2115  6187]
 [ 8192    13  1424 11674]]
Classification report : 
               precision    recall  f1-score   support

          1       0.63      0.86      0.72     24204
          2       0.47      0.03      0.06      1631
          3       0.47      0.17      0.25     12111
          4       0.55      0.55      0.55     21303

avg / total       0.56      0.59      0.55     59249



In [ ]:
tree.export_graphviz(xgboost_model,out_file="tree.txt",impurity = True,
                            feature_names = list(X.columns),
                            class_names = ['Indemne', 'Tué','BlesseHospitalisé','Blesséléger'],
                            rounded = True,
                            filled= True )tree.export_graphviz(dt,out_file="tree.txt",impurity = True,
                            feature_names = list(X.columns),
                            class_names = ['Indemne', 'Tué','BlesseHospitalisé','Blesséléger'],
                            rounded = True,
                            filled= True )

Résultats de xgboost avec getdummies est beaucoup plus meilleurs que sans getdummies.
Ceci est important à comprendre: en effet toutes nos variables- à part qui sont numériques- sont nominales, donc le fait de laisser un codage en entier pour chaque variable ne donne pas des résultats optimale vu que l'algorithme va les considérer comme étant des variables ordianales.

In [190]:
from collections import  Counter
Counter(list(y_test))


Counter({1: 24204, 2: 1631, 3: 12111, 4: 21303})

In [172]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [173]:
# Confusion matrix with random forest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
#x,y = final_data.loc[:,final_data.columns != 'grav'], final_data.loc[:,'grav']
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.3,random_state = 1)
rf = RandomForestClassifier(random_state = 4)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)
print('Classification report: \n',classification_report(y_test,y_pred))


Confusion matrix: 
 [[19852    45  1148  3159]
 [  312   154   769   396]
 [ 2460   222  4706  4723]
 [ 5964    90  3935 11314]]
Classification report: 
              precision    recall  f1-score   support

          1       0.69      0.82      0.75     24204
          2       0.30      0.09      0.14      1631
          3       0.45      0.39      0.42     12111
          4       0.58      0.53      0.55     21303

avg / total       0.59      0.61      0.60     59249



In [174]:
rf.feature_importances_
#l'age est important pour déterminer la gravité des accidents  (résultats du feature engineering)

array([ 0.07491471,  0.02161795,  0.00995786,  0.02140831,  0.01737334,
        0.03932014,  0.02373142,  0.03947745,  0.01627951,  0.03085286,
        0.04362372,  0.03262579,  0.0163669 ,  0.01592698,  0.01728349,
        0.06826311,  0.0154436 ,  0.01175445,  0.0297301 ,  0.00861901,
        0.00037745,  0.07667873,  0.00130716,  0.03048254,  0.03838609,
        0.04603939,  0.01636648,  0.03073791,  0.01814418,  0.02773542,
        0.03824001,  0.00756619,  0.01940826,  0.09395949])

In [306]:
# 

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.3,random_state = 1)
dt = DecisionTreeClassifier(random_state = 4,max_depth=6)
dt.fit(x_train,y_train)
y_pred = dt.predict(x_test)
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)
print('Classification report: \n',classification_report(y_test,y_pred))


Confusion matrix: 
 [[20938     1   391  2874]
 [  614    45   440   532]
 [ 4034    37  1723  6317]
 [ 8598     8  1315 11382]]
Classification report: 
              precision    recall  f1-score   support

          1       0.61      0.87      0.72     24204
          2       0.49      0.03      0.05      1631
          3       0.45      0.14      0.22     12111
          4       0.54      0.53      0.54     21303

avg / total       0.55      0.58      0.53     59249



In [304]:
from sklearn.grid_search import  GridSearchCV


In [315]:
from sklearn.grid_search import  GridSearchCV
dt = DecisionTreeClassifier(random_state = 4)
grid_values = {'max_depth': [3,4,5,6,7,8,9,10,],'min_samples_split':[5,10,15]}
#metric to optimize over grid parameters: Recall
grid_clf_recall = GridSearchCV(dt, param_grid = grid_values,scoring='accuracy')
grid_clf_recall.fit(x_train, y_train)

print(grid_clf_recall.best_score_)
print(grid_clf_recall.best_params_)
#y_decision_fn_scores_recall = grid_clf_recall.predict(X_test) 

#print('Test set Recall: ', recall_score(y_test, y_decision_fn_scores_recall))
#print('Grid best parameter (max. Recall): ', grid_clf_recall.best_params_)
#print('Grid best score (Recall): ', grid_clf_recall.best_score_)

0.5815099061824126
{'max_depth': 10, 'min_samples_split': 15}


In [316]:
dt=grid_clf_recall.best_estimator_
dt.fit(x_train,y_train)
print('test accuracy:',dt.score(x_test,y_test))

test accuracy: 0.581815726848


In [296]:
from sklearn import tree
with open("tree1.dot","w") as f :
    tree.export_graphviz(dt,out_file=f,impurity = True,
                            feature_names = list(X.columns),
                            class_names = ['Indemne', 'Tué','BlesseHospitalisé','Blesséléger'],
                            rounded = True,
                            filled= True )

In [297]:
tree.export_graphviz(dt,out_file="tree.txt",impurity = True,
                            feature_names = list(X.columns),
                            class_names = ['Indemne', 'Tué','BlesseHospitalisé','Blesséléger'],
                            rounded = True,
                            filled= True )